# Part 3: Estimating inundation extent using HAND methodology


In this example we will use some of GRASS GIS hydrology tools, namely:

* [r.watershed](https://grass.osgeo.org/grass-stable/manuals/r.watershed.html): for computing flow accumulation, drainage direction, the location of streams and watershed basins; it does not need sink filling because of using the least-cost-path to route flow out of sinks
* [r.lake](https://grass.osgeo.org/grass-stable/manuals/r.lake.html): to fill a lake to a target water level from a given start point or seed raster
* [r.lake.series](https://grass.osgeo.org/grass-stable/manuals/r.lake.series.html): addon which runs r.lake for different water levels
* [r.stream.distance](https://grass.osgeo.org/grass-stable/manuals/r.stream.distance.html): for computing the distance to streams or outlet, the relative elevation above streams; the distance and the elevation are calculated along watercourses

First, let's create a new mapset *flooding* in nc_spm_08_grass7 sample dataset:

In [ ]:
%%bash
grass -c -e ~/grassdata/nc_spm_08_grass7/flooding

Initialize GRASS session:

In [ ]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
gj.init(os.path.expanduser("~/grassdata"), "nc_spm_08_grass7", "flooding")

Modules r.stream.distance and r.lake.series are addons and we need to install them first:

In [ ]:
gs.run_command("g.extension", extension="r.stream.distance")
gs.run_command("g.extension", extension="r.lake.series")

### Compute HAND raster

We will estimate inundation extent using the Height Above Nearest Drainage methodology ([A.D. Nobre, 2011](https://doi.org/10.1016/j.jhydrol.2011.03.051)). We will compute the HAND terrain model representing the differences in elevation between each grid cell and the elevations of the flowpath-connected downslope grid cells where the flow enters the channel.

First we compute the flow accumulation, drainage and streams (with a threshold value of 100000). We convert the streams to vector for better visualization.

In [ ]:
gs.run_command("g.region", raster="elevation")
gs.run_command("r.watershed", elevation="elevation", accumulation="flowacc", drainage="drainage", stream="streams", threshold=100000)
gs.run_command("r.to.vect", input="streams", output="streams", type="line")

img = gj.GrassRenderer()
img.d_rast(map="flowacc")
img.d_vect(map="streams", width=2, color="blue")
img.d_legend(raster="flowacc", range="0,1000")
img.show()

Let's zoom in to see the flow accumulation raster better:

In [ ]:
fig = gj.InteractiveMap(width=600)
fig.add_raster("flowacc")
fig.add_vector("streams")
fig.show()

Now we use r.stream.distance with output parameter difference to compute a new raster map where each cell is the elevation difference between the cell and the the cell on the stream where the cell drains.

In [ ]:
gs.run_command("r.stream.distance", stream_rast="streams", direction="drainage", elevation="elevation", method="downstream", difference="above_stream")
gs.run_command("r.colors", map="above_stream", color="elevation")

img = gj.GrassRenderer()
img.d_rast(map="above_stream")
img.d_legend(raster="above_stream")
img.show()

### Inundation
Before we compute the inundation, we will look at how r.lake works. We compute a lake from a specified coordinate pair and water level:

In [ ]:
gs.run_command("r.lake", elevation="elevation", water_level=90, lake="lake", coordinates=[637877, 218475])

img = gj.GrassRenderer()
img.d_rast(map="elevation")
img.d_rast(map="lake")
img.d_legend(raster="lake", label_values="0.1,5,10,15", digits=0)
img.show()

Now instead of the elevation raster we use the HAND raster to simulate 5-meter inundation and, as the seed we specify the entire stream.

In [ ]:
gs.run_command("r.lake", elevation="above_stream", water_level=5, lake="flood", seed="streams")

img = gj.GrassRenderer()
img.d_rast(map="above_stream")
img.d_rast(map="flood")
img.d_legend(raster="flood", label_values="0.1,4,8", digits=0)
img.show()

With the r.lake.series addon we can create a series of inundation maps with rising water levels:

In [ ]:
gs.run_command("r.lake.series", elevation="above_stream", start_water_level=0, end_water_level=5, water_level_step=0.5, output="inundation", seed_raster="streams")
gs.run_command("t.rast.colors", input="inundation", color="water")
print(gs.read_command("t.rast.list", input="inundation"))

r.lake.series creates a space-time dataset. We can use the [temporal modules](https://grass.osgeo.org/grass-stable/manuals/temporal.html) to further work with the data. For example, we could further compute the volume and extent of flood water using t.rast.univar:

In [ ]:
import pandas as pd
data = gs.read_command("t.rast.univar", input="inundation", separator="comma")
pd.DataFrame([line.split(",") for line in data.splitlines()])

### Visualization

Here we show how to create a GIF animation of the raising water levels using Python. First, we render individual images in a loop with increasing inundation level:

In [ ]:
from PIL import Image

images = []
for raster in gs.list_grouped(type="raster", pattern="inundation_*")["flooding"]:
    img = gj.GrassRenderer()
    img.d_rast(map="elevation_shade")
    img.d_vect(map="streets_wake", color="#4D4D4D")
    img.d_rast(map=raster)
    f = img.filename
    images.append(Image.open(img.filename))

Now create an animated GIF:

In [ ]:
from grass.imaging import writeGif       
from IPython.display import Image

writeGif(filename="inundation.gif", images=images, duration=0.2, repeat=True)
Image("inundation.gif")

We can do the same with the *Animation Tool* in GUI:

![Using the Animation Tool for animating inundation](img/flooding.gif)